# exploratory analysis examining at day to day trends 
### Jan 5, 2023

In [22]:
import json
import datetime
import pandas as pd
import numpy as np
import math
import sys
import os
from collections import OrderedDict

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import seaborn as sns
import seaborn.objects as so
# Apply the default theme
sns.set_theme()
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, "../scripts")
from helpers import *

In [5]:
df = pd.read_csv("../constants/td_ambient_102022.csv" )
df["DateTime"] = pd.to_datetime(df["DateTime"])
df.head()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1


## comparison of point values

In [7]:
dftime = df.copy()

# transform datetime index to actual pandas datetime, and set as index of dataframe 
dftime["DateTime"] = pd.to_datetime(dftime["DateTime"])
dftime.set_index("DateTime", inplace=True)


In [8]:

with open('../constants/window_treatment.json') as f:    
    window_treatment = json.load(f)

exp_a_end = str2dt(window_treatment["072522"]["cutoff_times"]["end"])
exp_b_end = str2dt(window_treatment["081622"]["cutoff_times"]["end"])

# only consider experiment a 
df_a = df.loc[df["DateTime"] < exp_a_end]
df_a.head()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1


In [11]:
df1, df0 = [x.reset_index(drop=True) for _, x in df_a.groupby(df_a['Room'] < 1)]
df0_days = [group[1] for group in df0.groupby(df0["DateTime"].dt.date)]
df1_days = [group[1] for group in df1.groupby(df1["DateTime"].dt.date)]


In [19]:
df1["DateTime"].iloc[0].date()

datetime.date(2022, 7, 24)

In [23]:
rmse_days_dict = {}
for df0, df1 in zip(df0_days, df1_days):
    n = 30
    a = len(df1)/n
    eff_d = {}
    delta_d = {}

    for i in range(int(a)):
        # print(i)
        k = n*i
        arr1 = df1["Temp C"].iloc[k:k+n]
        arr2 = df0["Temp C"].iloc[k:k+n]
        eff_size = cohend(arr1, arr2)
        assert df1["DateTime"].iloc[k] == df0["DateTime"].iloc[k]
        eff_d[df1["DateTime"].iloc[k]] = eff_size
        delta_d[df1["DateTime"].iloc[k]] = rmse(arr1, arr2)

    eff_df = pd.DataFrame(eff_d, index=[0]).T 
    rmse_df = pd.DataFrame(delta_d, index=[0]).T 
    rmse_days_dict[df1["DateTime"].iloc[0].date()]  = rmse_df

rmse_days_dict = OrderedDict(rmse_days_dict)

In [24]:
len(rmse_days_dict)

5

In [32]:
rmse_df.index.time

array([datetime.time(0, 0), datetime.time(0, 15), datetime.time(0, 30),
       datetime.time(0, 45), datetime.time(1, 0), datetime.time(1, 15),
       datetime.time(1, 30), datetime.time(1, 45), datetime.time(2, 0),
       datetime.time(2, 15), datetime.time(2, 30), datetime.time(2, 45),
       datetime.time(3, 0), datetime.time(3, 15), datetime.time(3, 30),
       datetime.time(3, 45), datetime.time(4, 0), datetime.time(4, 15),
       datetime.time(4, 30), datetime.time(4, 45), datetime.time(5, 0),
       datetime.time(5, 15), datetime.time(5, 30), datetime.time(5, 45),
       datetime.time(6, 0), datetime.time(6, 15), datetime.time(6, 30),
       datetime.time(6, 45), datetime.time(7, 0)], dtype=object)

In [33]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=True,)

for df0, df1, rmse_df, ix in zip(df0_days, df1_days, rmse_days_dict.values(), range(len(rmse_days_dict))):
    fig.add_trace(go.Scatter(
    x=rmse_df.index.time,
    y=rmse_df[0],
    name="RMSE of Temperature in Rooms (15 min intervals)"
    ), row=ix+1, col=1)

fig.show()


In [68]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=rmse_df.index,
    y=rmse_df[0],
    name="RMSE of Temperature in Rooms (15 min intervals)"
))

fig.add_trace(go.Scatter(
    x=df0["DateTime"],
    y=df0["Window Open"],
    name="Room 0 Window Open"
))

fig.add_trace(go.Scatter(
    x=df1["DateTime"],
    y=df1["Window Open"],
    name="Room 1 Window Open"
))

fig.add_trace(go.Scatter(
    x=df0["DateTime"],
    y=normalize(df0["Temp C"]),
    name="Normalized Room 0 Temp"
))

fig.add_trace(go.Scatter(
    x=df1["DateTime"],
    y=normalize(df1["Temp C"]),
    name="Normalized Room 1 Temp"
))




fig.update_layout(title='Temperature, RMSE Comparison between Rooms',
                   xaxis_title='Dates',
                   yaxis_title='RMSE')

## comparison of averages 

In [41]:
df_15min = make_df_with_freq(df, "15T")
df_15min.head()

Temp C       RH %  Room  Ambient Temp  Ambient RH  \
Room DateTime                                                                  
0    2022-07-20 07:15:00  23.1806  51.421100   0.0     14.333333   30.555556   
     2022-07-20 07:30:00  23.3258  51.759333   0.0     14.333333   30.555556   
     2022-07-20 07:45:00  23.4242  51.619233   0.0     14.333333   30.555556   
     2022-07-20 08:00:00  23.5314  51.218100   0.0     14.666667   30.555556   
     2022-07-20 08:15:00  23.6314  50.950367   0.0     14.666667   30.555556   

                          Window Open  
Room DateTime                          
0    2022-07-20 07:15:00          1.0  
     2022-07-20 07:30:00          1.0  
     2022-07-20 07:45:00          1.0  
     2022-07-20 08:00:00          1.0  
     2022-07-20 08:15:00          1.0